In [1]:
import os
os.makedirs('/content/drive', exist_ok=True)

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!unzip './drive/MyDrive/cv_competition_data.zip'

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: cv_competition_data/trainset/A_ZLcqQq2f4nQAAAAAAAAAAAAAAQAAAQ_aug2.jpg  
  inflating: cv_competition_data/trainset/A_ZLiKSpkyGXoAAAAAAAAAAAAAAQAAAQ.jpg  
  inflating: cv_competition_data/trainset/A_ZLiKSpkyGXoAAAAAAAAAAAAAAQAAAQ_aug1.jpg  
  inflating: cv_competition_data/trainset/A_ZlJJTLDKz78AAAAAAAAAAAAAAQAAAQ.jpg  
  inflating: cv_competition_data/trainset/A_ZlJJTLDKz78AAAAAAAAAAAAAAQAAAQ_aug1.jpg  
  inflating: cv_competition_data/trainset/A_ZlJJTLDKz78AAAAAAAAAAAAAAQAAAQ_aug2.jpg  
  inflating: cv_competition_data/trainset/A_ZlJUTZFVALcAAAAAAAAAAAAAAQAAAQ.jpg  
  inflating: cv_competition_data/trainset/A_ZlJUTZFVALcAAAAAAAAAAAAAAQAAAQ_aug1.jpg  
  inflating: cv_competition_data/trainset/A_zlRmQalzEREAAAAAAAAAAAAAAQAAAQ.jpg  
  inflating: cv_competition_data/trainset/A_zlRmQalzEREAAAAAAAAAAAAAAQAAAQ_aug1.jpg  
  inflating: cv_competition_data/trainset/A_zlRmQalzEREAAAAAAAAAAAAAAQAAAQ_aug2.jpg  
  inflating: cv_competition_data/trainset/A_zLs

In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

import time
import os
import shutil
from PIL import Image
from tqdm import tqdm

In [7]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, mode, transform=None):
        """
        Args:
            root_dir (string): 项目根目录。
            mode (string): 模式，包含train, validate, test
            transform (callable, optional): 可选的转换操作。
        """
        if transform is None:
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
            ])

        self.root_dir = root_dir
        self.transform = transform

        self.annotation_path = os.path.join(root_dir, f'annotations/{"val" if mode == "test" else "train"}.txt')
        self.img_path_dir = os.path.join(root_dir, f'{"val" if mode == "test" else "train"}set')

        self.samples = self.get_annotations(mode, self.annotation_path)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_name, label = self.samples[idx]
        img_path = os.path.join(self.img_path_dir, img_name).replace("*", "_")

        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return (img_name, image), label

    def get_annotations(self, mode, annotation_path):
        image_labels = []
        with open(annotation_path, 'r') as file:
            for line in file:
                img_name, label = line.strip().split()
                image_labels.append((img_name, int(label)))

        count = len(image_labels)
        # count = 100
        if mode == "validate":
            image_labels = image_labels[:count//10]
        elif mode == "train":
            image_labels = image_labels[count//10:count]

        return image_labels

def get_dataset(root_dir, mode):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    dataset = CustomDataset(root_dir, mode, transform)
    return dataset


def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg,(1, 2, 0)))
    plt.show()

Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of image names: ('A_IWYPR4p7uvzoJ5BKu4so_AAAAQAAAQ_aug1.jpg', 'A_easkS5V9IcQAAAAAAAAAAAAAAQAAAQ_aug2.jpg', 'A_EMJVTpF9ZhIAAAAAAAAAAAAAAQAAAQ_aug1.jpg', 'A*ywMwTKblb94AAAAAAAAAAAAAAQAAAQ.jpg', 'A_VKkzToIs2-IAAAAAAAAAAAAAAQAAAQ_aug1.jpg', 'A_6oFjQ5QHMs0AAAAAAAAAAAAAAQAAAQ_aug2.jpg', 'A*49TOToJFqEoAAAAAAAAAAAAAAQAAAQ.jpg', 'A_hSfRRK4Tpg8AAAAAAAAAAAAAAQAAAQ_aug1.jpg', 'A_mG60QaD8EmkAAAAAAAAAAAAAAQAAAQ_aug1.jpg', 'A_oiUlTKol-vAAAAAAAAAAAAAAAQAAAQ_aug2.jpg', 'A*jlCxTJ2N1ysAAAAAAAAAAAAAAQAAAQ.jpg', 'A_CG23TY0mXMwAAAAAAAAAAAAAAQAAAQ_aug1.jpg', 'A_CMcLSrSJ8cgAAAAAAAAAAAAAAQAAAQ_aug1.jpg', 'A__bMtSbw_FtP5dzZDg5-49AAAAQAAAQ_aug2.jpg', 'A_3s9TTofR7rIAAAAAAAAAAAAAAQAAAQ_aug2.jpg', 'A_ub_XSaf1grcAAAAAAAAAAAAAAQAAAQ_aug1.jpg', 'A_L-sQTJ6RFd0AAAAAAAAAAAAAAQAAAQ_aug2.jpg', 'A_MkhVSJsPFTgAAAAAAAAAAAAAAQAAAQ_aug1.jpg', 'A_riGhQb_ZLIkAAAAAAAAAAAAAAQAAAQ_aug2.jpg', 'A_LRIdTpb2A7Z18hdCthcIdwAAAQAAAQ_aug1.jpg', 'A_4spVQ5Ky6x0AAAAAAAAAAAAAAQAAAQ_aug2.

Labels:
 A_RyqjSpjWzUoAAAAAAAAAAAAAAQAAAQ_aug2.jpg:  42;
A*J6XXRZqiKxkAAAAAAAAAAAAAAQAAAQ.jpg:  12;
A_tRLSRLT5Fo0AAAAAAAAAAAAAAQAAAQ_aug2.jpg:  16;
A_j0n5SorGkIAAAAAAAAAAAAAAAQAAAQ_aug2.jpg:  53;
A*cCn5QoMLkc8AAAAAAAAAAAAAAQAAAQ.jpg:  14;
A_y7dpRZimrR0AAAAAAAAAAAAAAQAAAQ_aug1.jpg:  61;
A*BeFhS74V3-IAAAAAAAAAAAAAAQAAAQ.jpg:  12;
A*TfcRQI--wXoAAAAAAAAAAAAAAQAAAQ.jpg:  40;
A*MfW1T5YhzscAAAAAAAAAAAAAAQAAAQ.jpg:  79;
A*d8ulRKFApNYAAAAAAAAAAAAAAQAAAQ.jpg:  29;
A_XNIhTLfsyBEAAAAAAAAAAAAAAQAAAQ_aug1.jpg:   3;
A*D3WbTb9STZ8AAAAAAAAAAAAAAQAAAQ.jpg:  87;
A*vwNDQJs-y9EAAAAAAAAAAAAAAQAAAQ.jpg:  65;
A*cOieTa0TqiYAAAAAAAAAAAAAAQAAAQ.jpg:  94;
A__DXLTLW8nhJWL4tLkeoqFgAAAQAAAQ_aug1.jpg:  52;
A_jUVzS43__hwAAAAAAAAAAAAAAQAAAQ_aug2.jpg:  88;
A*dsqqTbnu4r0AAAAAAAAAAAAAAQAAAQ.jpg:  26;
A_2PKCS5eDctwAAAAAAAAAAAAAAQAAAQ_aug1.jpg:  33;
A_ub_XSaf1grcAAAAAAAAAAAAAAQAAAQ_aug1.jpg:  50;
O1CN01PwvDfN1PFU0ylVbdF_!!6000000001811-0-antinsbiocamera_aug1.jpg:  23;
A*CaF4QIN_hwwAAAAAAAAAAAAAAQAAAQ.jpg:  27;
A_6V_eSpeTdiw

In [8]:
# 定义转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 项目根目录
root_dir = '/content/cv_competition_data'

# 创建数据集和数据加载器
dataset = CustomDataset(root_dir=root_dir, mode="train", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 遍历数据并打印图像和标签的形状
for (names, images), labels in dataloader:
    print(f'Batch of images shape: {images.shape}')  # 打印图像的形状
    print(f'Batch of image names: {names}')  # 打印标签
    print(f'Batch of labels: {labels}')  # 打印标签
    break  # 只打印一个批次的数据以进行测试

# get some random training images
dataiter = iter(dataloader)
(names, images), labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
# print labels
print('Labels:\n', '\n'.join('%20s: %3d;' % (names[j], labels[j]) for j in range(32)))

In [9]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # 输入通道为3 (RGB)，输出通道为32
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # 下采样一半
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), # 卷积层
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)                  # 再次下采样
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),                                         # 展平
            nn.Linear(64 * 56 * 56, 128),                        # 全连接层
            nn.ReLU(),
            nn.Linear(128, 1)                                    # 最终输出单个值（预测年龄）
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


class MyResNetModel(nn.Module):
    def __init__(self):
        super(MyResNetModel, self).__init__()
        resnet = models.resnet50(pretrained=True)

        for param in resnet.parameters():
            param.requires_grad = False

        self.resnet = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(resnet.fc.in_features, 1)

    def forward(self, x):
        x = self.resnet(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


class MyResNetModel2(nn.Module):
    def __init__(self):
        super(MyResNetModel2, self).__init__()
        resnet = models.resnet50(pretrained=True)

        for param in resnet.parameters():
            param.requires_grad = False

        for param in resnet.layer4.parameters():
            param.requires_grad = True

        self.resnet = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(resnet.fc.in_features, 1)

    def forward(self, x):
        x = self.resnet(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [10]:
def train(device, train_loader, validate_loader, model, optimizer, criterion, max_epochs, save_path):
    """
    Train the model for regression with early stopping based on validation loss.

    Args:
        device (torch.device): The device to use for training (e.g., "cuda" or "cpu").
        train_loader (DataLoader): DataLoader for the training dataset.
        validate_loader (DataLoader): DataLoader for the validation dataset.
        model (nn.Module): The model to train.
        optimizer (torch.optim.Optimizer): Optimizer for model training.
        criterion (nn.Module): Loss function for regression (e.g., MSELoss).
        max_epochs (int): Maximum number of epochs to train.
        save_path (str): Directory to save the best model.
    """
    best_loss = float('inf')
    patience = 3  # Stop if no improvement for 3 consecutive epochs
    patience_counter = 0

    for epoch in range(max_epochs):
        model.train()
        running_loss = 0.0

        # Training loop
        for (names, images), labels in tqdm(train_loader, desc="Epoch_" + str(epoch) + " Train Processing:"):
            images, labels = images.to(device), labels.to(device).float()

            # Forward pass
            outputs = model(images).squeeze(1)  # Ensure outputs are 1D
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{max_epochs}], Train Loss: {avg_train_loss:.4f}")

        # Validation loop
        model.eval()
        validate_loss = 0.0

        with torch.no_grad():
            for (names, images), labels in validate_loader:
                images, labels = images.to(device), labels.to(device).float()

                outputs = model(images).squeeze(1)
                loss = criterion(outputs, labels)
                validate_loss += loss.item()

        avg_validate_loss = validate_loss / len(validate_loader)
        print(f"Validation Loss: {avg_validate_loss:.4f}")

        is_best = False
        # Check for improvement
        if avg_validate_loss < best_loss:
            print("Validation loss improved, saving model...")
            is_best = True
            best_loss = avg_validate_loss
            patience_counter = 0
        else:
            patience_counter += 1
        save_checkpoint(model.state_dict(), save_path, is_best, epoch)

        # Early stopping
        if patience_counter >= patience:
            print("Early stopping triggered. Best validation loss:", best_loss)
            break

    print("Training complete. Best validation loss:", best_loss)



def save_checkpoint(state, save_root, is_best, epoch):
    if not os.path.exists(save_root):
        os.makedirs(save_root)
    save_path = os.path.join(save_root, 'epoch_{}.pth.tar'.format(str(epoch)))
    torch.save(state, save_path)

    best_path = os.path.join(save_root, 'best_model.pth.tar'.format(str(epoch)))
    torch.save(state, best_path)

In [12]:
print("Start...")
root_dir = '/content/cv_competition_data'
batch_size = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyResNetModel2().to(device)

criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_dataset = get_dataset(root_dir, "train")
validate_dataset = get_dataset(root_dir, "validate")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, drop_last=False)
validate_loader = torch.utils.data.DataLoader(validate_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, drop_last=False)

train(
    device,
    train_loader,
    validate_loader,
    model,
    optimizer,
    criterion,
    max_epochs=50,
    save_path=os.path.join(root_dir, "save")
)

Start...


Epoch_0 Train Processing:: 100%|██████████| 1408/1408 [10:42<00:00,  2.19it/s]


Epoch [1/50], Train Loss: 1221.7459
Validation Loss: 1346.7094
Validation loss improved, saving model...


Epoch_1 Train Processing:: 100%|██████████| 1408/1408 [10:11<00:00,  2.30it/s]


Epoch [2/50], Train Loss: 1056.6718
Validation Loss: 1710.8093


Epoch_2 Train Processing:: 100%|██████████| 1408/1408 [09:46<00:00,  2.40it/s]


Epoch [3/50], Train Loss: 889.0447
Validation Loss: 1660.3107


Epoch_3 Train Processing:: 100%|██████████| 1408/1408 [09:40<00:00,  2.42it/s]


Epoch [4/50], Train Loss: 619.1380
Validation Loss: 1849.0896
Early stopping triggered. Best validation loss: 1346.7094427218103
Training complete. Best validation loss: 1346.7094427218103


In [13]:
def test(device, test_loader, model, criterion):
    """
    Evaluate the model on the test set.

    Args:
        device (torch.device): The device to use for testing (e.g., "cuda" or "cpu").
        test_loader (DataLoader): DataLoader for the test dataset.
        model (nn.Module): The trained model.
        criterion (nn.Module): Loss function for regression (e.g., MSELoss).

    Returns:
        float: The average test loss.
    """
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    total_samples = 0
    total_correct = 0  # For accuracy or other metrics if needed

    with torch.no_grad():  # No need to track gradients during testing
        with open('output.txt', 'w', encoding='utf-8') as file:
            for (names, images), labels in tqdm(test_loader, desc="Eval Processing:"):
                images, labels = images.to(device), labels.to(device).float()

                # Forward pass
                outputs = model(images).squeeze(1)  # Ensure outputs are 1D

                for i, name in enumerate(names):
                    file.write(f"{name}\t{round(outputs[i].item())}\n")

                loss = criterion(outputs, labels)

                test_loss += loss.item()

                # If you want to compute additional metrics, like MSE or MAE, you can do so here
                # For example, to compute MAE:
                total_samples += len(labels)
                total_correct += torch.abs(outputs - labels).sum().item()  # Absolute error

    avg_test_loss = test_loss / len(test_loader)
    avg_mae = total_correct / total_samples  # Mean Absolute Error

    print(f"Test Loss: {avg_test_loss:.4f}")
    print(f"Test MAE: {avg_mae:.4f}")

    return avg_test_loss, avg_mae

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
root_dir = '/content/cv_competition_data'
batch_size = 32
model = MyResNetModel2()

# 2. 加载模型权重
model_path = os.path.join(root_dir, "save/best_model.pth.tar")  # 模型保存路径
model.load_state_dict(torch.load(model_path))  # 加载权重到模型
model.to(device)

test_dataset = get_dataset(root_dir, "test")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, drop_last=False)

criterion = torch.nn.MSELoss()

# 调用测试函数
avg_test_loss, avg_mae = test(device, test_loader, model, criterion)
print(avg_test_loss, avg_mae)

<ipython-input-15-60e8beec4951>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # 加载权重到模型
Eval Processing:: 100%|██████████| 

Test Loss: 1072.8603
Test MAE: 23.5209
1072.8602781904506 23.520858194986978
